In [0]:
import pandas as pd
import numpy as np
import pickle
import os
import glob
from tqdm import tqdm

In [0]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
jpgpaths=glob.glob("/content/drive/My Drive/stanfordAI/***.jpg")
jpgpaths.sort()

In [10]:
jpg_names=[]
for jpg in tqdm(jpgpaths):
    jpg_names.append(os.path.basename(jpg).split('.')[0])
len(jpg_names)
print(jpg_names[0:10])

100%|██████████| 21004/21004 [00:00<00:00, 578471.51it/s]

['0005b896-33ad-4ecf-93b4-dad735ad69b60001', '0005b896-33ad-4ecf-93b4-dad735ad69b60002', '0005b896-33ad-4ecf-93b4-dad735ad69b60003', '0005b896-33ad-4ecf-93b4-dad735ad69b60004', '0005b896-33ad-4ecf-93b4-dad735ad69b60005', '0005b896-33ad-4ecf-93b4-dad735ad69b60006', '0005b896-33ad-4ecf-93b4-dad735ad69b60007', '0005b896-33ad-4ecf-93b4-dad735ad69b60008', '0005b896-33ad-4ecf-93b4-dad735ad69b60009', '0005b896-33ad-4ecf-93b4-dad735ad69b60010']


In [54]:
jpg = pd.DataFrame(jpg_names,columns=['id'])
jpg['name'] = jpg['id'].apply(lambda x: x[0:-4])
jpg['second'] = jpg['id'].apply(lambda x: x[-4:])
jpg = jpg[jpg.second != '%04d'] #filter out those ending with '%04d'
jpg['second'] = jpg['second'].astype('float')
jpg.head()
#jpg.to_csv('jpg_names.csv')
#!cp jpg_names.csv drive/My\ Drive/

,id,name,second
0,0005b896-33ad-4ecf-93b4-dad735ad69b60001,0005b896-33ad-4ecf-93b4-dad735ad69b6,1.0
1,0005b896-33ad-4ecf-93b4-dad735ad69b60002,0005b896-33ad-4ecf-93b4-dad735ad69b6,2.0
2,0005b896-33ad-4ecf-93b4-dad735ad69b60003,0005b896-33ad-4ecf-93b4-dad735ad69b6,3.0
3,0005b896-33ad-4ecf-93b4-dad735ad69b60004,0005b896-33ad-4ecf-93b4-dad735ad69b6,4.0
4,0005b896-33ad-4ecf-93b4-dad735ad69b60005,0005b896-33ad-4ecf-93b4-dad735ad69b6,5.0


In [0]:
def mapping(filepath):
    data = dict()
    df = pd.read_csv(filepath)
    if len(df)>0:
      cols = ['Time','AU02', 'AU04', 'AU05', 'AU17','AU18','AU26','Unilateral_LAU12','Unilateral_RAU12']
      df = df.loc[:,df.columns.isin(cols)]
      missing_col = list(set(cols)-set(df.columns))
      if len(missing_col) > 0: 
          for col in missing_col: df[col] = None
      name = os.path.basename(filepath[:-10])
      jpg_df = jpg[jpg.name==name]
      df_merge = pd.merge_asof(jpg_df,df,left_on='second',right_on='Time',direction='backward')
      for i in range(df_merge.shape[0]):
          data[df_merge.loc[i,'id']]=list(df_merge.loc[i,['AU02', 'AU04', 'AU05', 'AU17','AU18','AU26','Unilateral_LAU12','Unilateral_RAU12']])
      return data
    else:
      return None



In [0]:
filepaths = glob.glob("/content/drive/My Drive/AMFEDPLUS.zip (Unzipped Files)/AMFEDPLUS_Distribution/AU Labels/***.csv")
filepaths.sort()

In [119]:
results = dict()
for filepath in tqdm(filepaths):
    print(os.path.basename(filepath[:-10]))
    result = mapping(filepath)
    if result is not None:
      results.update(result)


  1%|          | 4/545 [00:00<00:19, 27.43it/s]

0005b896-33ad-4ecf-93b4-dad735ad69b6
0013e1aa-5ba4-440e-b4b0-2fbd46e9807f
0040b0de-783a-47f3-bf38-59902d929a65
012d7ad7-71af-41fa-9a10-4b49feb6e00b
016d17e7-a947-4273-8aec-dee6f99a716b



  1%|▏         | 7/545 [00:00<00:21, 25.61it/s]

019e963b-2e9d-4182-93b8-c20308595983
020e95e0-8a56-4038-87e6-faf484c2802a
02453a81-1994-44c7-8669-a831a1be6d22
02615018-5620-46b9-8eec-da3009eb2d9c
02b068a8-67c9-4c75-be2f-5fd3f62cbb36



  3%|▎         | 14/545 [00:00<00:19, 26.62it/s]

...


100%|█████████▉| 543/545 [01:40<00:00,  2.50it/s]

feb598a5-d07c-4ff3-8d32-f8cc6831eb61



100%|█████████▉| 544/545 [01:41<00:00,  2.52it/s]

ff99a4d9-3c7d-4593-850c-fe8db5097cab



100%|██████████| 545/545 [01:41<00:00,  2.49it/s]


In [122]:
results['ff99a4d9-3c7d-4593-850c-fe8db5097cab0001']

[0.0, 0.0, 0.0, None, 0.0, None, 0.0, 0.0]

In [0]:
with open('aus.pkl', 'wb') as f:
    pickle.dump(results, f)
!cp aus.pkl drive/My\ Drive/